#### 📊 GPT-2 Small (117M) — Parameter Count Breakdown

##### Model Specs
- **12 decoder blocks**
- **12 attention heads per block**
- **Hidden size:** 768  
- **FFN size:** 3072  
- **Vocabulary:** 50,257  
- **Context length:** 1024  

---

#### 🧱 1. Embedding Layer

##### Token Embedding
```
50,257 × 768 = 38,610,816 params
```

##### Positional Embedding
```
1024 × 768 = 786,432 params
```

###### ✔️ Total Embeddings = ~39.4M

---

#### 🔁 2. Decoder Block (×12)

Each of the 12 blocks contains:

---

##### 🧲 A. Masked Multi-Head Attention

###### Q, K, V projection matrices
```
3 × (768 × 768) = 1,769,472
biases ≈ 2,304
```

###### Output projection
```
768 × 768 = 589,824
bias ≈ 768
```

###### ✔️ Attention per block ≈ 2.36M

---

##### ⚙️ B. Feed-Forward Network (FFN)

###### Linear 1 (768 → 3072)
```
768 × 3072 = 2,359,296
bias = 3072
```

###### Linear 2 (3072 → 768)
```
3072 × 768 = 2,359,296
bias = 768
```

###### ✔️ FFN per block ≈ 4.72M

---

##### 📏 C. LayerNorms
Each block has 2 LayerNorms:
```
2 × (768 + 768) = 3072 params
```

###### ✔️ LayerNorm per block ≈ 0.003M

---

##### 📦 Total per Decoder Block
```
Attention:   ~2.36M
FFN:         ~4.72M
LayerNorms:   0.003M
--------------------------------
Total/block ≈ 7.083M
```

---

#### 🧮 3. Total for 12 Blocks
```
12 × 7.083M = 84.996M ≈ 85M
```

---

#### 🔚 4. Output Layer (LM Head)
GPT-2 uses **weight tying** → LM head shares weights with token embeddings.  
✔️ No extra parameters

---

#### 🎯 Final Parameter Count
```
Embeddings:   ~39.4M
12 Blocks:    ~85.0M
-------------------------
Total:       ~124M
```

###### ⚠️ Official number: **117M**  
Differences come from:
- tied embeddings  
- compression/optimization  
- rounding in documentation  

---

#### 📘 Summary Table

| Component | Params |
|----------|--------|
| Token Embeddings | 38.6M |
| Positional Embeddings | 0.79M |
| Multi-Head Attention | 28.3M |
| Feed-Forward | 56.6M |
| LayerNorm | 0.04M |
| **Total (Actual)** | **~124M** |
| **GPT-2 Small (Official)** | **117M** |




In [ ]:
!pip install tiktoken

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import tiktoken

# -----------------------------
# 1. PARAMETERS
# -----------------------------
batch_size = 8              # keep small for GPU memory, increase later
max_length = 1024
stride = 100
embedding_dim = 768
num_heads = 12
mlp_hidden_dim = 3072
num_layers = 12              # GPT-2 small = 12 layers, reduce for demo
lr = 3e-4
epochs = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------
# 2. READ RAW TEXT
# -----------------------------
with open("./the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# -----------------------------
# 3. TOKENIZER
# -----------------------------
tokenizer = tiktoken.get_encoding("gpt2")
tokens = tokenizer.encode(raw_text)
vocab_size = tokenizer.n_vocab

# -----------------------------
# 4. SLIDING WINDOW DATASET
# -----------------------------
class SlidingWindowDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, max_length, stride):
        self.tokens = tokens
        self.max_length = max_length
        self.stride = stride

        self.inputs = []
        self.targets = []

        i = 0
        while i + max_length < len(tokens):
            seq_in = tokens[i:i+max_length]
            seq_out = tokens[i+1:i+max_length+1]
            self.inputs.append(seq_in)
            self.targets.append(seq_out)
            i += stride

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs[idx], dtype=torch.long), torch.tensor(self.targets[idx], dtype=torch.long)

dataset = SlidingWindowDataset(tokens, max_length, stride)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# -----------------------------
# 5. TRANSFORMER BLOCK
# -----------------------------
class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, mlp_hidden_dim):
        super().__init__()
        self.ln1 = nn.LayerNorm(embedding_dim)
        self.ln2 = nn.LayerNorm(embedding_dim)
        self.attn = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Linear(mlp_hidden_dim, embedding_dim)
        )

    def forward(self, x):
        # Self-attention with residual
        x_norm = self.ln1(x)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x = x + attn_out

        # Feed-forward with residual
        x_norm = self.ln2(x)
        x = x + self.mlp(x_norm)
        return x

# -----------------------------
# 6. GPT-2 Style Model
# -----------------------------
class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, max_length, embedding_dim, num_heads, mlp_hidden_dim, num_layers):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = nn.Embedding(max_length, embedding_dim)
        self.blocks = nn.ModuleList([
            TransformerBlock(embedding_dim, num_heads, mlp_hidden_dim) for _ in range(num_layers)
        ])
        self.ln_f = nn.LayerNorm(embedding_dim)
        self.lm_head = nn.Linear(embedding_dim, vocab_size, bias=False)

    def forward(self, input_ids):
        batch, seq_len = input_ids.shape
        positions = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        x = self.token_embedding(input_ids) + self.pos_embedding(positions)

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits

# -----------------------------
# 7. TRAINING
# -----------------------------
model = SimpleGPT(vocab_size, max_length, embedding_dim, num_heads, mlp_hidden_dim, num_layers)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_idx, (input_ids, target_ids) in enumerate(dataloader):
        input_ids = input_ids.to(device)
        target_ids = target_ids.to(device)

        optimizer.zero_grad()
        logits = model(input_ids)                # (batch, seq, vocab_size)
        loss = criterion(logits.view(-1, vocab_size), target_ids.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1} finished, Avg Loss: {total_loss/len(dataloader):.4f}")

# -----------------------------
# 8. SAVE MODEL LOCALLY
# -----------------------------
torch.save(model.state_dict(), "simple_gpt_model.pth")
print("Model saved as simple_gpt_model.pth")


Epoch 1, Batch 0, Loss: 10.9896
Epoch 1 finished, Avg Loss: 9.2457
Epoch 2, Batch 0, Loss: 7.8389
Epoch 2 finished, Avg Loss: 6.9950
Epoch 3, Batch 0, Loss: 6.1085
Epoch 3 finished, Avg Loss: 5.6197
Epoch 4, Batch 0, Loss: 4.9615
Epoch 4 finished, Avg Loss: 4.6463
Epoch 5, Batch 0, Loss: 4.1953
Epoch 5 finished, Avg Loss: 3.9221
Epoch 6, Batch 0, Loss: 3.5472
Epoch 6 finished, Avg Loss: 3.3249
Epoch 7, Batch 0, Loss: 2.9555
Epoch 7 finished, Avg Loss: 2.7789
Epoch 8, Batch 0, Loss: 2.5989
Epoch 8 finished, Avg Loss: 2.3812
Epoch 9, Batch 0, Loss: 2.0161
Epoch 9 finished, Avg Loss: 2.0621
Epoch 10, Batch 0, Loss: 1.7500
Epoch 10 finished, Avg Loss: 1.7780
Epoch 11, Batch 0, Loss: 1.5158
Epoch 11 finished, Avg Loss: 1.5352
Epoch 12, Batch 0, Loss: 1.4968
Epoch 12 finished, Avg Loss: 1.3012
Epoch 13, Batch 0, Loss: 1.2358
Epoch 13 finished, Avg Loss: 1.1061
Epoch 14, Batch 0, Loss: 0.9104
Epoch 14 finished, Avg Loss: 0.9164
Epoch 15, Batch 0, Loss: 0.8266
Epoch 15 finished, Avg Loss: 0.73

In [ ]:
# Recreate model architecture (must match the saved model)
model = SimpleGPT(vocab_size, max_length, embedding_dim, num_heads, mlp_hidden_dim, num_layers)
model.load_state_dict(torch.load("simple_gpt_model.pth", map_location=device))
model.to(device)
model.eval()  # important for inference (disables dropout etc.)


SimpleGPT(
  (token_embedding): Embedding(50257, 768)
  (pos_embedding): Embedding(1024, 768)
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=768, out_features=3072, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=3072, out_features=768, bias=True)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
prompt = "She raised her eyebrows"
input_ids = torch.tensor([tokenizer.encode(prompt)], device=device)  # shape: (1, seq_len)


In [ ]:
generated = input_ids.clone()

num_generate = 5
for _ in range(num_generate):
    with torch.no_grad():
        logits = model(generated)             # (1, seq, vocab_size)
        next_token_logits = logits[:, -1, :]  # take last token
        next_token_id = torch.argmax(next_token_logits, dim=-1, keepdim=True)  # greedy
        generated = torch.cat((generated, next_token_id), dim=1)


In [ ]:
generated_text = tokenizer.decode(generated[0].tolist())
print("Generated text:\n", generated_text)


Generated text:
 She raised her eyebrows with its _famille
